# WeatherPy
----

### Analysis
* As expected, the weather becomes significantly warmer as one approaches the equator (0 Deg. Latitude). More interestingly, however, is the fact that the southern hemisphere tends to be warmer this time of year than the northern hemisphere. This may be due to the tilt of the earth.
* There is no strong relationship between latitude and cloudiness. However, it is interesting to see that a strong band of cities sits at 0, 80, and 100% cloudiness.
* There is no strong relationship between latitude and wind speed. However, in northern hemispheres there is a flurry of cities with over 20 mph of wind.

---

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime
# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
#Samples are uniformly distributed over the half-open interval [low, high) (includes low, but excludes high)
#size : Output shape: int or tuple of ints, optional
# test size = 150 , actual size 1500, reduces to about 600 unqiue values
lats = np.random.uniform(low=-90.000, high=90.000, size=15)
lngs = np.random.uniform(low=-180.000, high=180.000, size=15)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
cityCount=len(cities)

In [3]:
#Prepare containers (City	Cloudiness	Country	Date	Humidity	Lat	Lng	Max Temp	Wind Speed)
rcities=[]
rlats=[]
rlngs=[]
clouds=[]
countries=[]
dates=[]
humidities=[]
mxTemps=[]
winds=[]
degs=[]

In [ ]:
### Perform API Calls
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"  # not metric
appid = api_key
query_url = url+"appid="+api_key+"&units="+units+"&q="

cityCounter=1
for city in cities:
    try:
        city_query_url = query_url+city 
        print(city_query_url)
        res = requests.get(city_query_url)
        print=(response)
        response=re.json()
        rcities.append(response['name'])
        rlats.append(response['coord']['lat'])
        rlngs.append(response['coord']['lon'])
        clouds.append(response['clouds']['all'])
        countries.append(response['sys']['country'])
        dates.append(response['dt'])
        humidities.append(response['main']['humidity'])
        mxTemps.append(response['main']['temp_max'])
        winds.append(response['wind']['speed'])
        degs.append(response['wind']['deg'])
        print("Processing Record "+str( cityCounter)+ " of " +str(cityCount)+" | " + city)
        cityCounter= cityCounter+1
    except:
        print('Skipped '+city)
#print(response)


http://api.openweathermap.org/data/2.5/weather?appid=8a5956e8e6e2385d1df09a3af9d9d728&units=Imperial&q=atuona
Skipped atuona
http://api.openweathermap.org/data/2.5/weather?appid=8a5956e8e6e2385d1df09a3af9d9d728&units=Imperial&q=hovd
Skipped hovd
http://api.openweathermap.org/data/2.5/weather?appid=8a5956e8e6e2385d1df09a3af9d9d728&units=Imperial&q=puerto ayora
Skipped puerto ayora
http://api.openweathermap.org/data/2.5/weather?appid=8a5956e8e6e2385d1df09a3af9d9d728&units=Imperial&q=albany
Skipped albany
http://api.openweathermap.org/data/2.5/weather?appid=8a5956e8e6e2385d1df09a3af9d9d728&units=Imperial&q=kyra
Skipped kyra
http://api.openweathermap.org/data/2.5/weather?appid=8a5956e8e6e2385d1df09a3af9d9d728&units=Imperial&q=dustlik


In [ ]:
print(len(rcities), len(countries), 
      len(dates),len(rlats),len(rlngs),len(clouds),len(humidities),len(mxTemps),len(winds), len(degs))

In [ ]:
### Convert Raw Data to DataFrame & save
weather_dict = {
    "City": rcities,
    "Country":countries,
    "Date":dates,
    "Lat": rlats,
    "Long": rlngs,
    "Cloudiness": clouds,
    "Humidity": humidities,
    "Max Temp.":mxTemps,
    "Wind Speed": winds,
    "Wind Direction":degs
}

weather_data = pd.DataFrame(weather_dict)
now = datetime.now()

filen="weatherData"+now.strftime("%Y%m%d") +".csv"
weather_data.to_csv(filen)

weather_data.head()

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
import seaborn as sns
sns.set()

# x: lat , y: temp
x= weather_data['Lat']
y= weather_data['Max Temp.']
s= y*10
plt.figure(figsize=(14, 7), dpi= 72)

plt.scatter(x,y, marker='o', facecolors='y' ,edgecolors='tab:orange', alpha=0.75)
#gca=get current axis
plt.gca().set(xlim=(min(weather_data['Lat']-10),max(weather_data['Lat']+10)), 
              ylim=(min(weather_data['Max Temp.'])-5,max(weather_data['Max Temp.'])+10), 
              xlabel='Latitude', 
              ylabel='Temperature (•F)')
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title='Temperature (F) vs. Latitude '
plt.legend(fontsize=16)
plt.show()

#### Latitude vs. Humidity Plot

In [ ]:
x= weather_data['Lat']
y= weather_data['Humidity']
s= y*10
plt.figure(figsize=(14, 7), dpi= 72)

plt.scatter(x,y, marker='o', facecolors='tab:gray' ,edgecolors='b', alpha=0.75)
#gca=get current axis
plt.gca().set(xlim=(min(weather_data['Lat']-10),max(weather_data['Lat']+10)), 
              ylim=(0,105), 
              xlabel='Latitude', 
              ylabel='Humidity (%)')
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title='Humidity  vs. Latitude '
plt.legend(fontsize=16)
plt.show()

In [ ]:
x= weather_data['Lat']
y= weather_data['Cloudiness']
s= y*10
plt.figure(figsize=(14, 7), dpi= 72)

plt.scatter(x,y, marker='o', facecolors='b' ,edgecolors='tab:gray', alpha=0.75)
#gca=get current axis
plt.gca().set(xlim=(min(weather_data['Lat']-10),max(weather_data['Lat']+10)), 
              ylim=(-5,105), 
              xlabel='Latitude', 
              ylabel='Cloudiness (%)')
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title='Cloudiness  vs. Latitude '
plt.legend(fontsize=16)
plt.show()

 #Latitude vs. Wind Speed Plot

In [ ]:
#sns.unset()
colors=[]
markers=[]
mvalues=[]
for d in degs:
    if int(d) > 337 or int(d) < 22.5:
        markers.append('\\rightarrow')
        colors.append('tab:blue')
        mvalues[0]=d
    elif int(d) >=22.5 and int(d) <65:
        markers.append('\\nearrow')
        colors.append('tab:orange')
        mvalues[1]=d
    elif int(d) >=65 and int(d) < 112.5:
        markers.append('\\uparrow')
        colors.append('tab:green')
        mvalues[2]=d
    elif int(d) >=112.5 and int(d) < 157.5:
        markers.append('\\nwarrow')
        colors.append('tab:red')
        mvalues[3]=d
    elif int(d) >=157.5 and int(d) < 202.5:
        markers.append('\\leftarrow')
        colors.append('tab:purple')
        mvalues[4]=d
    elif int(d) >=202.5 and int(d) < 247.5:
        markers.append('\\swarrow')
        colors.append('tab:brown')
        mvalues[5]=d
    elif int(d) >=247.5 and int (d) < 292.5:
        markers.append('\\downarrow')
        colors.append('tab:pink')
        mvalues[6]=d
    else: 
        markers.append('\\searrow')
        colors.append('tab:olive')
        mvalues[7]=d
#print(markers)

avmarkers_df=pd.DataFrame(markers,degs).reset_index()

def getMarker(i):
    return "$"+markers[i% len(markers)]+"$"

def getColor(i):
    return colors[i% len(colors)]



In [ ]:


x= weather_data['Lat']
y= weather_data['Wind Speed']
s= y*10
plt.figure(figsize=(14, 7), dpi= 72)

#plt.scatter(x,y, marker=markers, facecolors='tab:gray' ,edgecolors='b', alpha=0.75)
for c, value in enumerate(markers):
    plt.plot(x[c],y[c], marker=getMarker(c), c=getColor(c), markersize=13)
#gca=get current axis
plt.gca().set(xlim=(min(weather_data['Lat']-10),max(weather_data['Lat']+10)), 
              ylim=(min(weather_data['Wind Speed']-5),max(weather_data['Wind Speed']+5)), 
              xlabel='Latitude', 
              ylabel='Wind Speed (mph)')
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.title='Wind Speed vs. Latitude '
#import matplotlib as mpl
#mpl.rcParams['legend.numpoints'] = 1

plt.legend(mvalues,fontsize=16)
plt.show()